In [0]:
%cd drive/My\ Drive/social_distancing
!ls

In [0]:
import numpy as np
import cv2
from scipy.spatial import distance as dist
import imutils
from google.colab.patches import cv2_imshow
import os

In [0]:
print(cv2.__version__)

In [0]:
!pip install --upgrade opencv-python

In [0]:
def detect_humans(frame,model,layer,min_confidence,nms_threshold,person_idx=0):
    (h,w)=frame.shape[:2]
    results=[]

    blob=cv2.dnn.blobFromImage(frame,1/255.0,(416,416),swapRB=True,crop=False)
    model.setInput(blob)
    outputs=model.forward(layer)

    bboxes=[]
    centroids=[]
    confidence_scores=[]

    for output in outputs:
        for detection in output:
            scores=detection[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]

            if class_id==person_idx and confidence>min_confidence:
                bbox=detection[0:4]*np.array([w,h,w,h])
                (centre_x,centre_y,width,height)=bbox.astype('int')
                x=int(centre_x-(width/2))
                y=int(centre_y-(height/2))
                bboxes.append([x,y,int(width),int(height)])
                centroids.append((centre_x,centre_y))
                confidence_scores.append(float(confidence))
    
    idxs=cv2.dnn.NMSBoxes(bboxes,confidence_scores,min_confidence,nms_threshold)
    if len(idxs)>0:
        for i in idxs.flatten():
            (x,y)=(bboxes[i][0],bboxes[i][1])
            (w,h)=(bboxes[i][2],bboxes[i][3])
            r=(confidence_scores[i],(x,y,x+w,y+h),centroids[i])
            results.append(r)
    return results

In [0]:
model_path='./yolo'
min_conf=0.3
nms_threshold=0.3
min_dist=50
input_video='pedestrians.mp4'
output_dir=os.getcwd()

In [0]:
labels_path=os.path.sep.join([model_path,'coco.names'])
labels=open(labels_path).read().strip().split('\n')
weights_path=os.path.sep.join([model_path,'yolov3.weights'])
config_path=os.path.sep.join([model_path,'yolov3.cfg'])

In [0]:
model=cv2.dnn.readNetFromDarknet(config_path,weights_path)
model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [0]:
layers=model.getLayerNames()
layer=[layers[i[0]-1] for i in model.getUnconnectedOutLayers()]
vs=cv2.VideoCapture(input_video)
writer=None
while True:
    (grabbed,frame)=vs.read()
    if not grabbed:
        break
    
    frame=imutils.resize(frame,width=700)
    results=detect_humans(frame,model,layer,min_conf,nms_threshold,person_idx=labels.index('person'))

    sd_violate=set()
    if len(results)>=2:
        centroids=np.array([r[2] for r in results])
        d=dist.cdist(centroids,centroids,metric="euclidean")
        for i in range(0,d.shape[1]):
            for j in range(i+1,d.shape[1]):
                if d[i,j]<min_dist:
                    sd_violate.add(i)
                    sd_violate.add(j)
    
    for(i,(prob,bbox,centroid)) in enumerate(results):
        (start_x,start_y,end_x,end_y)=bbox
        (c_x,c_y)=centroid
        colour=(0,255,0)
        if i in sd_violate:
            colour=(0,0,255)
        cv2.rectangle(frame,(start_x,start_y),(end_x,end_y),colour,2)
        cv2.circle(frame,(c_x,c_y),5,colour,1)
    
    cv2_imshow(frame)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

    fourcc=cv2.VideoWriter_fourcc(*"MJPG")
    writer=cv2.VideoWriter(output_dir,fourcc,25,(frame.shape[1],frame.shape[0]),True)

    if writer is not None:
        writer.write(frame)